In [41]:
import numpy as np
from numpy import linalg as LA
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

def GetEigen_WithWeights (Prov_mat, Cons_mat, Prov_Weight, Cons_Weight):
    dim  = int(math.sqrt(Prov_mat.size))
    union_mat = Prov_mat*Prov_Weight + Cons_mat*Cons_Weight
    row_sum = np.sum(union_mat,axis = 1) #functionals sum
    col_sum = np.sum(union_mat,axis = 0) #structors sum
    deg_mat = np.diag(np.concatenate((row_sum,col_sum))) #functionals first
    adj_mat = np.block([[np.zeros((dim,dim)),union_mat], [union_mat.T, np.zeros((dim,dim))]])
    laplacian = np.matrix(deg_mat-adj_mat)
    w,v = LA.eig(laplacian)
    #print(np.around(v,4),np.around(w,4))
    fiedler_idx = np.argsort(w)[1]
    #print(Cons_Weight,fiedler_idx,v)
    fiedler_value = w[fiedler_idx]
    fiedler_vector = v[:,fiedler_idx]
    return union_mat,laplacian, fiedler_value,fiedler_vector

In [42]:
Provider_weight = 1
Consumer_weights = [1.2, 1.5, 2, 4, 8, 16, 32, 64]

## observer

In [43]:
provider_str = """
1	0	0	0	0	0	0   0
1	1	0	0	0	0	0   0
0	1	1	0	0	0	0   0
0	0	0	1	0	0	0   0
0	0	0	1	1	0	0   0
0	0	0	0	0	1	0   0
0	0	0	0	0	0	1   0
0	0	0	0	0	1	1   1
"""

consumer_str = """
0	0	0	0	0	0	0   0
0	0	1	0	0	0	0   0
0	0	0	1	0	0	0   0
0	0	0	0	0	0	0   1
1	0	0	0	0	0	0   0
0	0	0	0	0	0	0   0
0	0	0	0	0	0	0   0
0	0	0	0	0	0	0   0
"""

mat = np.fromstring(provider_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
provider_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

mat = np.fromstring(consumer_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
consumer_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

In [48]:
f_vals = []
f_vecs = []
for cons_w in Consumer_weights:
    union_mat,laplacian,f_val,f_vec = GetEigen_WithWeights(provider_mat,consumer_mat,Provider_weight,cons_w)
    #print(cons_w,f_vec)
    f_vals.append(f_val)
    f_vecs.append(f_vec.A1.tolist())
    if cons_w==2:
        ### display laplacian for weight 2
        indx = ['F{}'.format(i) for i in range(1, dim+1)]
        header = ['S{}'.format(i) for i in range(1, dim+1)]
        union_df = pd.DataFrame(union_mat, columns= header, index = indx)
        u=union_df.style.background_gradient(cmap='Wistia').set_caption("UNION")
        laplacian_df = pd.DataFrame(laplacian, columns= indx+header, index = indx+header)
        lap=laplacian_df.style.background_gradient(cmap='Wistia').set_caption("LAPLACIAN (D-A)")
        display(u)
        display(lap)
        ####

,S1,S2,S3,S4,S5
F1,1,1,0,0,2
F2,0,1,0,0,0
F3,0,2,1,1,0
F4,0,0,0,1,0
F5,0,0,0,0,1


,F1,F2,F3,F4,F5,S1,S2,S3,S4,S5
F1,4,0,0,0,0,-1,-1,0,0,-2
F2,0,1,0,0,0,0,-1,0,0,0
F3,0,0,4,0,0,0,-2,-1,-1,0
F4,0,0,0,1,0,0,0,0,-1,0
F5,0,0,0,0,1,0,0,0,0,-1
S1,-1,0,0,0,0,1,0,0,0,0
S2,-1,-1,-2,0,0,0,4,0,0,0
S3,0,0,-1,0,0,0,0,1,0,0
S4,0,0,-1,-1,0,0,0,0,2,0
S5,-2,0,0,0,-1,0,0,0,0,3


In [45]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'orange' if val < 0 else 'green'
    return 'background-color: %s' % color
vheader = ['Weight {}: '.format(i) + str(x) for i,x in zip(Consumer_weights,np.around(f_vals,3))]
header = ['S{}'.format(i) for i in range(1, dim+1)]
indx = ['F{}'.format(i) for i in range(1, dim+1)]
res_df = pd.DataFrame(np.around(f_vecs,5).T, columns= vheader, index = indx+header)
pd.options.display.max_colwidth = 130
res=res_df.style.\
    applymap (color_negative_red).\
    set_caption("Observer Fiedler EIGENVALUES & EIGENVECTORS")
#display(res) 

## Command

In [46]:
provider_str = """
1	1	0	0	0
0	1	0	0	0
0	0	1	1	0
0	0	0	1	0
0	0	0	0	1
"""
consumer_str = """
0	0	0	0	1
0	0	0	0	0
0	1	0	0	0
0	0	0	0	0
0	0	0	0	0
"""
mat = np.fromstring(provider_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
provider_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

mat = np.fromstring(consumer_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
consumer_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

f2_vals = []
f2_vecs = []
for cons_w in Consumer_weights:
    laplacian,f_val,f_vec = GetEigen_WithWeights(provider_mat,consumer_mat,Provider_weight,cons_w)
    #print(cons_w,f_vec)
    f2_vals.append(f_val)
    f2_vecs.append(f_vec.A1.tolist())
    if cons_w==2:
        ### display laplacian for weight 2
        indx = ['F{}'.format(i) for i in range(1, dim+1)]
        header = ['S{}'.format(i) for i in range(1, dim+1)]
        laplacian_df = pd.DataFrame(laplacian, columns= indx+header, index = indx+header)
        lap=laplacian_df.style.background_gradient(cmap='Wistia').set_caption("LAPLACIAN (D-A)")
        display(lap)
        ####

vheader = ['Weight {}: '.format(i) + str(x) for i,x in zip(Consumer_weights,np.around(f2_vals,3))]
header = ['S{}'.format(i) for i in range(1, dim+1)]
indx = ['F{}'.format(i) for i in range(1, dim+1)]
res_df = pd.DataFrame(np.around(f2_vecs,5).T, columns= vheader, index = indx+header)
pd.options.display.max_colwidth = 130
res1=res_df.style.\
    applymap (color_negative_red).\
    set_caption("Command Fiedler EIGENVALUES & EIGENVECTORS")

ValueError: too many values to unpack (expected 3)

In [0]:
ax = plt.gca()
df = pd.DataFrame({
    'Weights':Consumer_weights,
    'Observer Fiedler values':f_vals,
    'Command Fiedler values':f2_vals})
df.plot(kind='line',x='Weights',y='Observer Fiedler values',color='red', ax=ax)
df.plot(kind='line',x='Weights',y='Command Fiedler values',color='blue', ax=ax)
display(res) 
display(res1) 
display(f_vals)
display(f2_vals)
